In [1]:
import os

In [2]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [11]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class DataValidationConfig:
    """
    Configuration class for data validation.

    Attributes:
        root_dir (Path): The root directory where data validation operations will take place.
        STATUS_FILE (str): The name of the status file that will store the status of the data validation process.
        unzip_data_dir (Path): The directory where the unzipped data is stored.
        all_schema (dict): A dictionary containing all the schema definitions for the data to be validated.
    """
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts

    def get_data_validation_config(self) -> DataValidationConfig:
        """
        Get the DataValidationConfig object from the configuration.
        
        Returns:
            DataValidationConfig: Configuration for data validation.
        """
        config = self.config.data_validation  # Get the data validation configuration section
        schema = self.schema.COLUMNS  # Get the schema columns

        create_directories([config.root_dir])  # Ensure the root directory for data validation exists

        # Create and return a DataValidationConfig object with the configuration details
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config


In [14]:
import os
from mlProject import logger

In [15]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        """
        Initialize the DataValidation class with a configuration.
        
        Args:
            config (DataValidationConfig): Configuration for data validation.
        """
        self.config = config  # Store the configuration

    def validate_all_columns(self) -> bool:
        """
        Validate that all columns in the dataset match the expected schema and their data types are correct.
        
        Returns:
            bool: True if all columns and their data types are valid, False otherwise.
        """
        try:
            validation_status = True  # Initialize validation status
            
            # Read the CSV file from the unzipped data directory
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)  # Get the list of columns in the data

            # Get the list of expected columns and their data types from the schema
            all_schema = self.config.all_schema

            # Validate each column in the data
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False  # Set validation status to False if a column is not in the schema
                    logger.error(f"Column {col} is not in the schema")
                else:
                    # Check the data type of the column
                    expected_dtype = all_schema[col]
                    actual_dtype = str(data[col].dtype)
                    if actual_dtype != expected_dtype:
                        validation_status = False  # Set validation status to False if data type does not match
                        logger.error(f"Data type for column {col} does not match. Expected: {expected_dtype}, Found: {actual_dtype}")

            # Write the final validation status to the status file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status  # Return the final validation status
        
        except Exception as e:
            raise e  # Raise the exception if any error occurs


In [16]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Retrieve data validation configuration
    data_validation_config = config.get_data_validation_config()
    
    # Initialize DataValidation with the retrieved configuration
    data_validation = DataValidation(config=data_validation_config)
    
    # Validate all columns in the dataset
    data_validation.validate_all_columns()
except Exception as e:
    # Raise the exception if any error occurs
    raise e


[2024-06-13 10:57:54,971: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-13 10:57:54,973: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-13 10:57:54,975: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-13 10:57:54,976: INFO: common: Created directory at: artifacts]
[2024-06-13 10:57:54,977: INFO: common: Created directory at: artifacts/data_validation]
